In [1]:
from pymongo import MongoClient
import os, sys, re
from nltk import FreqDist
import pandas as pd
import datetime

In [2]:
client = MongoClient("bioresearch-d1:23001")
db = client.WATSON

In [3]:
interventions_without_id = []
for study in db.CTGOV.find({"metadata.STATUS":"PENDING"}):
    temp = [ x["NAME"] for x in study["interventions"] if "ID" not in x ]
    interventions_without_id.extend(temp)
    
fdist = FreqDist(interventions_without_id)
print fdist.most_common()[:10]
df = pd.DataFrame(fdist.most_common(), columns=["Term", "Count"])
print df[:10]
#df.to_excel("/Users/singhv/Desktop/trial_stats/common_uncurated_terms.xlsx", index=False)

[(u'Laboratory Biomarker Analysis', 363), (u'Pharmacological Study', 121), (u'laboratory biomarker analysis', 109), (u'Quality-of-Life Assessment', 65), (u'Placebo', 50), (u'Radiation Therapy', 29), (u'Intensity-Modulated Radiation Therapy', 28), (u'pharmacological study', 27), (u'Therapeutic Conventional Surgery', 19), (u'radiation therapy', 18)]
                                    Term  Count
0          Laboratory Biomarker Analysis    363
1                  Pharmacological Study    121
2          laboratory biomarker analysis    109
3             Quality-of-Life Assessment     65
4                                Placebo     50
5                      Radiation Therapy     29
6  Intensity-Modulated Radiation Therapy     28
7                  pharmacological study     27
8       Therapeutic Conventional Surgery     19
9                      radiation therapy     18


In [4]:
print db.CTGOV.find({"$or": [{"interventions.NAME":"Laboratory Biomarker Analysis"}, {"interventions.NAME":"laboratory biomarker analysis"}]}).count()

676


In [5]:
lba_only_trials = []
top_10 = set([ x.lower() for (x, count) in fdist.most_common()[:10] ])
for study in db.CTGOV.find({"metadata.STATUS":"PENDING"}):
    temp = [ x["NAME"] for x in study["interventions"] if "ID" not in x ]
    #if (len(temp) == 1) and (temp[0].lower() in top_10):
    if set(temp).issubset(top_10):
        lba_only_trials.append(study["NCT_ID"])
        
print len(lba_only_trials)
#print "\n".join(lba_only_trials)

600


In [14]:
print type(series.value_counts())
df1 = pd.DataFrame(series.value_counts(), columns=["Count"])
print df1[:10]
print df1.to_excel("/Users/singhv/Desktop/trial_stats/common_uncurated_terms-1.xlsx")
#series.value_counts().to_excel("/Users/singhv/Desktop/trial_stats/common_uncurated_terms-1.xlsx")

<class 'pandas.core.series.Series'>
                                       Count
Laboratory Biomarker Analysis            363
Pharmacological Study                    121
laboratory biomarker analysis            109
Quality-of-Life Assessment                65
Placebo                                   50
Radiation Therapy                         29
Intensity-Modulated Radiation Therapy     28
pharmacological study                     27
Therapeutic Conventional Surgery          19
radiation therapy                         18
None


In [4]:
for row in df.itertuples():
    db.UNRESOLVED.insert_one({"NAME" : row[1], "LAST_UPDATED" : datetime.datetime.now(), "COUNT" : row[2] })